<a href="https://colab.research.google.com/github/raulbs7/Machine-Learning-Techniques-Project/blob/master/NLP_Supervised_Project/NLP_Hatred_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CLASSIFICATION PROJECT

In [ ]:
from google.colab import files
import pandas as pd
import nltk
import io
import re
import string
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize
try:
    import cPickle as pickle
except ImportError: 
    import pickle

import html
#import emo_unicode

nltk.download("popular")
nltk.download('vader_lexicon') #sentiment analysis
nltk.download('twython') #twitter

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

False

## Preprocessing

To preprocess the data, first it is needed to catch the data from the csv files.

In [ ]:
def upload_dataframes (index_fields):
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    df = pd.read_csv(io.StringIO(uploaded[fn].decode('utf-8')), index_col = index_fields)
    return df

In [ ]:
tweets = upload_dataframes([])
tweets.head()

Saving labeled_data.csv to labeled_data (1).csv
User uploaded file "labeled_data.csv" with length 2546446 bytes


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


One important thing is to remove symbols which are noise and not provide any information to the analysis of the text. The symbols that they are going to be removed are the next ones: ! " $ % & ( ) = _ * ^ ¡ @

In [ ]:
def remove_symbols(text):
  return re.sub('[!"\$%\&\(\)=_\*\ˆ¡@\|]', '', text)

In [ ]:
tweets['tweet'] = tweets['tweet'].apply(remove_symbols)
tweets.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,RT mayasolovely: As a woman you shouldn't com...
1,1,3,0,3,0,1,RT mleew17: boy dats cold...tyga dwn bad for ...
2,2,3,0,3,0,1,RT UrKindOfBrand Dawg RT 80sbaby4life: You ev...
3,3,3,0,2,1,1,RT CGAnderson: vivabased she look like a tranny
4,4,6,0,6,0,1,RT ShenikaRoberts: The shit you hear about me...


Next step in preprocessing is to remove the capital letters.

In [ ]:
def remove_capital_let(text):
  return text.lower()

In [ ]:
tweets['tweet'] = tweets['tweet'].apply(remove_capital_let)
tweets.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,rt mayasolovely: as a woman you shouldn't com...
1,1,3,0,3,0,1,rt mleew17: boy dats cold...tyga dwn bad for ...
2,2,3,0,3,0,1,rt urkindofbrand dawg rt 80sbaby4life: you ev...
3,3,3,0,2,1,1,rt cganderson: vivabased she look like a tranny
4,4,6,0,6,0,1,rt shenikaroberts: the shit you hear about me...


### LEMMATIZE ALL TERMS

In order to determinine the lemma of a word based on its intended meaning, the following step must be carried out:

1. Tokenization

In [ ]:
# 1. Tokenization
from nltk.tokenize import sent_tokenize, word_tokenize
def tokenization_words(raw):
  tokens = word_tokenize(raw)
  tokens
  return tokens

def tokenization_sent(raw):
  tokens = sent_tokenize(raw)
  tokens
  return tokens

In [ ]:
tweet_token = tweets['tweet'].apply(tokenization_words)

tweet_token.head()

0    [rt, mayasolovely, :, as, a, woman, you, shoul...
1    [rt, mleew17, :, boy, dats, cold, ..., tyga, d...
2    [rt, urkindofbrand, dawg, rt, 80sbaby4life, :,...
3    [rt, cganderson, :, vivabased, she, look, like...
4    [rt, shenikaroberts, :, the, shit, you, hear, ...
Name: tweet, dtype: object

In [ ]:
sentence_token = tweets['tweet'].apply(tokenization_sent)
sentence_token.head()

0    [ rt mayasolovely: as a woman you shouldn't co...
1    [ rt mleew17: boy dats cold...tyga dwn bad for...
2    [ rt urkindofbrand dawg rt 80sbaby4life: you e...
3    [ rt cganderson: vivabased she look like a tra...
4    [ rt shenikaroberts: the shit you hear about m...
Name: tweet, dtype: object

2. Stopwords

The stop words have to be filtered out before lemmatization. Stop words refers to the most common words in a language as prepositions or determiners.

In [ ]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

def stop_words(text):
  #STOPWORDS
  english_stops = set(stopwords.words('english'))
  clean_sent = [word for word in text if word not in english_stops]
  return clean_sent
  #STOPWORDS 

In [ ]:
tweet_clean_sent = tweet_token.apply(stop_words)
tweet_clean_sent.head()

0    [rt, mayasolovely, :, woman, n't, complain, cl...
1    [rt, mleew17, :, boy, dats, cold, ..., tyga, d...
2    [rt, urkindofbrand, dawg, rt, 80sbaby4life, :,...
3    [rt, cganderson, :, vivabased, look, like, tra...
4    [rt, shenikaroberts, :, shit, hear, might, tru...
Name: tweet, dtype: object

2. Lemmatization

In [ ]:
from nltk.stem import WordNetLemmatizer
def lemmatizer_words(text):
  lemmatizer = WordNetLemmatizer()
  return [lemmatizer.lemmatize(word) for word in text]

In [ ]:
tweet_lemmatizer=tweet_clean_sent.apply(lemmatizer_words)
tweet_lemmatizer.head()

0    [rt, mayasolovely, :, woman, n't, complain, cl...
1    [rt, mleew17, :, boy, dat, cold, ..., tyga, dw...
2    [rt, urkindofbrand, dawg, rt, 80sbaby4life, :,...
3    [rt, cganderson, :, vivabased, look, like, tra...
4    [rt, shenikaroberts, :, shit, hear, might, tru...
Name: tweet, dtype: object

### REMOVE CONTRACTIONS

Next, it can be optional to remove contractions to have a better tokenization.

In [ ]:
def remove_contractions(text):
  text = re.sub(r'isn\'t', r'is not', text)
  text = re.sub(r'aren\'t', r'are not', text)
  text = re.sub(r'wasn\'t', r'was not', text)
  text = re.sub(r'hadn\'t', r'had not', text)
  text = re.sub(r'haven\'t', r'have not', text)
  text = re.sub(r'wasn\'t', r'was not', text)
  text = re.sub(r'weren\'t', r'were not', text)
  text = re.sub(r'can\'t', r'can not', text)
  text = re.sub(r'doesn\'t', r'does not', text)
  text = re.sub(r'couldn\'t', r'could not', text)
  text = re.sub(r'shouldn\'t', r'should not', text)
  text = re.sub(r'don\'t', r'do not', text)
  text = re.sub(r'didn\'t', r'did not', text)
  text = re.sub(r'mustn\'t', r'must not', text)
  text = re.sub(r'might\'ve', r'might not', text)
  text = re.sub(r'won\'t', r'will not', text)
  text = re.sub(r'wouldn\'t', r'would not', text)
  text = re.sub(r'what\'s', r'what is', text)
  text = re.sub(r'where\'s', r'where is', text)
  text = re.sub(r'who\'s', r'who is', text)
  text = re.sub(r'who\'ll', r'who will', text)
  text = re.sub(r'that\'s', r'that is', text)
  text = re.sub(r'there\'s', r'there is', text)
  text = re.sub(r'i\'ll', r'i will', text)
  text = re.sub(r'i\'m', r'i am', text)
  text = re.sub(r'i\'have', r'i have', text)
  text = re.sub(r'i\'d', r'i would', text)
  text = re.sub(r'i\'d', r'i had', text)
  text = re.sub(r'you\'re', r'you are', text)
  text = re.sub(r'you\'ve', r'you have', text)
  text = re.sub(r'you\'ll', r'you will', text)
  text = re.sub(r'you\'d', r'you would', text)
  text = re.sub(r'you\'d', r'you had', text)
  text = re.sub(r'he\'s', r'he is', text)
  text = re.sub(r'he\'ll', r'he will', text)
  text = re.sub(r'she\'s', r'she is', text)
  text = re.sub(r'she\'ll', r'she will', text)
  text = re.sub(r'it\'s', r'it is', text)
  text = re.sub(r'it\'ll', r'it will', text)
  text = re.sub(r'we\'re', r'we are', text)
  text = re.sub(r'we\'ve', r'we have', text)
  text = re.sub(r'we\'ll', r'we will', text)
  text = re.sub(r'we\'d', r'we would', text)
  #text = re.sub(r'we\'d', r'we had', text)
  text = re.sub(r'they\'re', r'they are', text)
  text = re.sub(r'they\'ve', r'they have', text)
  text = re.sub(r'they\'ll', r'they will', text)
  text = re.sub(r'they\'d', r'they would', text)
  #text = re.sub(r'they\'d', r'they had', text)

  return text

In [ ]:
tweets['tweet'] = tweets['tweet'].apply(remove_contractions)
tweets.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,rt mayasolovely: as a woman you should not co...
1,1,3,0,3,0,1,rt mleew17: boy dats cold...tyga dwn bad for ...
2,2,3,0,3,0,1,rt urkindofbrand dawg rt 80sbaby4life: you ev...
3,3,3,0,2,1,1,rt cganderson: vivabased she look like a tranny
4,4,6,0,6,0,1,rt shenikaroberts: the shit you hear about me...


# Remove repeated words

In [ ]:
def remove_repeated(text):
  return re.sub(r'\b(\w+)( \1\b)+', r'\1', text)

There is an example to see how the previous function works because its complicated to show the effect in the dataset of tweets.

In [ ]:
print(remove_repeated("hello hello 1 1 how how are are you you ?"))

tweets['tweet'] = tweets['tweet'].apply(remove_repeated)
tweets.head()

hello 1 how are you ?


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,rt mayasolovely: as a woman you should not co...
1,1,3,0,3,0,1,rt mleew17: boy dats cold...tyga dwn bad for ...
2,2,3,0,3,0,1,rt urkindofbrand dawg rt 80sbaby4life: you ev...
3,3,3,0,2,1,1,rt cganderson: vivabased she look like a tranny
4,4,6,0,6,0,1,rt shenikaroberts: the shit you hear about me...


Due to the fact that tweets are being treated, there is a lot of probability to have some emojis in our data. These emojis are represented in our data as a code. Our goal, is to convert this emojis in tokenizable and meaningful words.

In [ ]:
def convert_emojis(text):
  text = html.unescape(text)
  for emot in emo_unicode.UNICODE_EMO:
    text = re.sub(re.escape('{}'.format(emot)), r'_'.join(emo_unicode.UNICODE_EMO[emot].replace(r",",r"").replace(r":",r"").split()), text)
  return text

In [ ]:
text = convert_emojis(text)
text

NameError: ignored

In [ ]:
tweets['tweet'] = tweets['tweet'].apply(correction)
tweets.head()

After that, it is necessary to correct wrong words of the text, in order to have a better analysis and to create a text sequence more understandable for our NLP algorithms.

In [ ]:
uploaded = files.upload()
file_name = "big.txt"
raw = uploaded[file_name].decode("utf-8")

In [ ]:
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(raw))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
#tweets['tweet'] = tweets['tweet'].apply(correction)
#tweets.head()

# Vectorization

0        [ rt mayasolovely: as a woman you shouldn't co...
1        [ rt mleew17: boy dats cold...tyga dwn bad for...
2        [ rt urkindofbrand dawg rt 80sbaby4life: you e...
3        [ rt cganderson: vivabased she look like a tra...
4        [ rt shenikaroberts: the shit you hear about m...
                               ...                        
24778    [you's a muthafin lie #8220;lifeasking: 20pear...
24779    [you've gone and broke the wrong heart baby, a...
24780    [young buck wanna eat.. dat nigguh like i aint...
24781              [youu got wild bitches tellin you lies]
24782    [~~ruffled  ntac eileen dahlia - beautiful col...
Name: tweet, Length: 24783, dtype: object

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfid_vectorizer(raw):
  vectorizer = TfidfVectorizer(min_df=1)
  return vectorizer.fit_transform(raw);

def tfidf_ngram__vectorizer(raw):
  vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=1)
  return vectorizer.fit_transform(raw);

#X = vectorizer.fit_transform(sent_tokenize)
tweet_vect = sentence_token.apply(tfid_vectorizer)

print(tweet_vect)
#print(tweet_vect.toarray())
print(vectorizer.vocabulary_)


0          (0, 6)\t0.3013403421812651\n  (0, 19)\t0.301...
1          (0, 12)\t0.25\n  (0, 0)\t0.25\n  (0, 14)\t0....
2          (0, 6)\t0.25394910913009533\n  (0, 14)\t0.25...
3          (0, 5)\t0.3779644730092272\n  (0, 1)\t0.3779...
4          (0, 0)\t0.17407765595569785\n  (0, 19)\t0.17...
                               ...                        
24778      (0, 3)\t0.2773500981126146\n  (0, 18)\t0.277...
24779      (0, 3)\t0.25\n  (0, 8)\t0.25\n  (0, 7)\t0.25...
24780      (0, 0)\t0.2886751345948129\n  (0, 9)\t0.2886...
24781      (0, 2)\t0.3779644730092272\n  (0, 5)\t0.3779...
24782      (0, 15)\t0.27735009811261463\n  (0, 0)\t0.27...
Name: tweet, Length: 24783, dtype: object
{'ruffled': 27, 'ntac': 19, 'eileen': 14, 'dahlia': 12, 'beautiful': 2, 'color': 8, 'combination': 10, 'of': 21, 'pink': 25, 'orange': 23, 'yellow': 30, 'amp': 0, 'white': 29, 'ruffled ntac': 28, 'ntac eileen': 20, 'eileen dahlia': 15, 'dahlia beautiful': 13, 'beautiful color': 3, 'color combination': 9, 'co

In [ ]:

tweet_vect = sentence_token.apply(tfidf_ngram__vectorizer)
#tfidf = X.toarray()
print(tweet_vect)
print(vectorizer.vocabulary_)
print (tweet_vect.shape)

pos_vocab = {v:i for i, v in enumerate(vectorizer.get_feature_names())}

#length
len(pos_vocab)


pos_vocab

#vectorizer.get_feature_names()

# KEYwORDS
#import numpy

#keywords = [vectorizer.get_feature_names()[numpy.argmax(x)] for x in X.toarray()]
# keywords